<a href="https://colab.research.google.com/github/BrotherKim/KAIST_CODE/blob/master/SEP531/term/4_KoBERT_emotion_finetune_jj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#변수 정의

In [1]:
ROOT_DIR = '/content/drive/MyDrive/KAIST/SEP531/kor_jj_labeled'
TRAIN_DIR = '%s/train' % ROOT_DIR
VALID_DIR = '%s/valid' % ROOT_DIR

NUM_TRAIN = 171
NUM_VALID = 58

SAVEPOINT_PATH = '/content/drive/MyDrive/KAIST/SEP531/KoBERT_emotion_finetuned_data_jj.pt'

#실행 환경#

- Python >= 3.6
- PyTorch >= 1.70
- Transformers = 3.0.2
- Colab
- batch size = 64 (convertable)
- epochs = 10 (convertable)

In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 1.7 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 12.8 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595744 sha256=2c5a7aa44b2dfad2e48b39670b989adc30164947496aeb90b540b8ea9e09978f
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 15.4 MB/s 
     |████████████████████████████████| 769 kB 13.1 MB/s 
     |████████████████████████████████| 895 kB 54.1 MB/s 
     |████████████████████████████████| 3.0 MB 49.1 MB/s 


#KoBERT 다운로드#

In [3]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-56wy_v84
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-56wy_v84
     |████████████████████████████████| 4.9 MB 13.0 MB/s 
     |████████████████████████████████| 3.3 MB 54.6 MB/s 
     |████████████████████████████████| 3.3 MB 55.5 MB/s 
     |████████████████████████████████| 61 kB 587 kB/s 
     |████████████████████████████████| 596 kB 47.6 MB/s 
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=13136 sha256=3ea89b3af44381ce45b233bdb1c38353605568c9d4ae0cd372597c0d80b02811
  Stored in directory: /tmp/pip-ephem-wheel-cache-448_spw5/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: tokenizers
    Found exist

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
#GPU 사용
device = torch.device("cuda:0")

In [7]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

.cache/kobert_v1.zip[██████████████████████████████████████████████████]
.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


#데이터셋 전처리#

In [8]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# 터미널 커맨드를 문자열로 반환해주는 함수 작성

import subprocess

def GetShellCmdStdOut(command):
  cmd = ['sh', '-c', command]
  fd_popen = subprocess.Popen(cmd, stdout=subprocess.PIPE).stdout 
  data = fd_popen.read().strip() 
  fd_popen.close()

  retval = data.decode('utf-8') 
  return retval

In [10]:
%pushd $TRAIN_DIR
tl = GetShellCmdStdOut('find | grep answer').split('\n')
train_json_list = [x.replace('./', '') for x in tl]
train_json_list.sort()
%popd

/content/drive/MyDrive/KAIST/SEP531/kor_jj_labeled/train
/content
popd -> /content


In [11]:
%pushd $VALID_DIR
vl = GetShellCmdStdOut('find | grep answer').split('\n')
valid_json_list = [x.replace('./', '') for x in vl]
valid_json_list.sort()
%popd

/content/drive/MyDrive/KAIST/SEP531/kor_jj_labeled/valid
/content
popd -> /content


In [12]:
train_json_list[0]

'DZJD20000949.json.answer'

In [13]:
!cat $TRAIN_DIR/{train_json_list[0]}

저 빙떡이 옥돔	저 빙떡이 솔라니	6
그 저 동천 사람들은 그렇게 말했다고 하더라고	그 저 동천 사람들은 경 골았댄 허드라고	1
성읍에서도 그렇게 말해 성읍에서도	성읍에서도 경 골아 성읍에서도	1
아 근데 우리는 뭐 중산간 동네에 사니까 우린	아 근디 우리는 뭐 중산간 동네에 살아노난 우린	3
한 나이 한 스무살 될때까지	혼 나이 혼 스무살 될때꼬지	1
옥돔이라고 한게 어디서 솔	솔라니엔 헌게 어디서 솔	1
옥돔이라고 (())못 먹어 봤어. {laughing}	솔라니랜 (())못 먹어 봐서. {laughing}	1
빙떡이야 뭐 뭐 뭐 빙떡이야 뭐 뭐 자꾸	빙떡사 미신 미신 뭐 빙떡사 미신 뭐 자꾸	6
뭐 일 있을때마다 먹었지만	뭐 일 이실때마다 먹어졌주만은	4
이거 옥돔이라고 한 것은 아니 진짜로 생각해보면은	이거 솔라니엔 헌 것은 아니 진짜로 생각해보면은	4
한 스무살 넘어야 이거 이런 물고기도 있었나 했던것 같아	혼 스무살 넘어사 이거 이런 괴기도 이서나신가 해진거 닮아	1
그런데 제사때도 뭐 전갱이 같은거 말렸다가	근디 식게때도 미신 각재기 고튼거 말리왔다그냉	4
올려진거 같고	올려진거 닮곡	1
에이 그래서 어?	에이 갠 어?	1
어렵긴 (()) 말해진것 같고	어렵긴 (()) 골아진거 닮고	4
그리고 또 동네마다 이렇게 같은 제일 에서도 이렇게	경허곡 또 동네마다 영 고튼 제일 에서도 영	1
달랐나 하는 생각이 드네	돌라나신가 허는 생각이 들어졈서	1
아이 저 중산간 마을 하고 해변 동네 하고 저 중산간 마을 하고	아이 저 웃드리 허고 해변 동네 허고게 저 웃드리 하고	4
뭐 옛날 그렇게 같이 안해 무슨	뭐 옛날 경 곧지 안해여 무슨	2
사돈 대접 한다고 해서 중산간 마을 사돈 오니까	사돈 대접 헌댄 해영 웃드리 사돈 오니까	4
해안 바닷가 동네에선 막 귀한거야 해서 뭐	해안 바당가 동네에선 막 귀한거여 해그냉 미신	4
물고기 한마리 사다가 뭐 대접 하면	괴기 혼마리 사당 미시거 대접 허면	4
이거 뭐 이렇게 저 (()) 하다가	이거 미시거 영 저 (()

In [14]:
import pandas as pd

chatbot_data = pd.read_csv('%s/%s' % (TRAIN_DIR, train_json_list[0]), sep='\t', header=None)

chatbot_data

,0,1,2
0,저 빙떡이 옥돔,저 빙떡이 솔라니,6
1,그 저 동천 사람들은 그렇게 말했다고 하더라고,그 저 동천 사람들은 경 골았댄 허드라고,1
2,성읍에서도 그렇게 말해 성읍에서도,성읍에서도 경 골아 성읍에서도,1
3,아 근데 우리는 뭐 중산간 동네에 사니까 우린,아 근디 우리는 뭐 중산간 동네에 살아노난 우린,3
4,한 나이 한 스무살 될때까지,혼 나이 혼 스무살 될때꼬지,1
...,...,...,...
380,조정에 바치니까,조정에 바치난게,4
381,벼슬도 주고 ((한목관)) 벼슬도 자자손손 하게끔 할 정도로,벼슬도 주고 ((한목관)) 벼슬도 자자손손 하게끔 할 정도로,4
382,말의 가치가 엄청 크니까,몰의 가치가 엄청 커노니까,4
383,응.,응.,4


In [15]:
def GenDataset(DIR, json_list):
  retval = []

  for fn in json_list:
    #print('%s\n' % (fn))
    fd = pd.read_csv('%s/%s' % (DIR, fn), sep='\t', header=None)
    # [1]데이터가 사투리, [2]가 감정
    for q, label in zip(fd[1], fd[2])  :
      #print('%s, %s\n' % (q, label))
      data = []
      data.append(q)
      data.append(str(label))
      retval.append(data)

  return retval

In [16]:
dataset_train = GenDataset(TRAIN_DIR, train_json_list)

In [17]:
dataset_valid = GenDataset(VALID_DIR, valid_json_list)

In [18]:
print(len(dataset_train))
print(len(dataset_valid))

35174
33703


In [19]:
dataset_train = dataset_train + dataset_valid[0:13000]
dataset_valid = dataset_valid[13001:]

dataset_train = dataset_train[0:45000]
dataset_valid = dataset_valid[0:15000]

In [22]:
print(len(dataset_train))
print(len(dataset_valid))

45000
15000


#KoBERT 입력 데이터로 변환#

In [23]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [24]:
# Setting parameters
max_len = 64
batch_size = 64 # origin : 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [25]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_valid = BERTDataset(dataset_valid, 0, 1, tok, max_len, True, False)

using cached model


In [26]:
# 첫번째는 패딩된 시퀀스, 두번째는 길이와 티입, 세번째는 어텐션 마스크 시퀀스
data_train[0]

(array([   2, 3990, 2551, 5970, 7096,  517, 6618, 6003, 5770,    3,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(10, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 6)

In [27]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_valid, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


#KoBERT 학습모델 만들기#

In [28]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [29]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/KAIST/SEP531/KoBERT_emotion_finetuned_data_added.pt'), strict=False)
model.eval()

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    

#KoBERT 모델 학습시키기#

In [30]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/704 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.3661649227142334 train acc 0.328125
epoch 1 batch id 201 loss 1.217261552810669 train acc 0.6333177860696517
epoch 1 batch id 401 loss 0.8553574681282043 train acc 0.6538731296758105
epoch 1 batch id 601 loss 1.1026588678359985 train acc 0.6528962146422629
epoch 1 train acc 0.6557839133522727


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.6849734042553192


  0%|          | 0/704 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.6553642749786377 train acc 0.765625
epoch 5 batch id 201 loss 0.410061240196228 train acc 0.929570895522388
epoch 5 batch id 401 loss 0.27188754081726074 train acc 0.943968204488778
epoch 5 batch id 601 loss 0.10789398103952408 train acc 0.9469893926788685
epoch 5 train acc 0.9494185014204546


  0%|          | 0/235 [00:00<?, ?it/s]

epoch 5 test acc 0.6890735815602836


In [31]:
torch.save(model.state_dict(), SAVEPOINT_PATH)